In [1]:
from math import isclose
from grgg import GRGG, Complementarity, Similarity, options

 # Tutorial

 This notebook provides a brief introduction to the `grgg` package
 implementing the Generalized Random Geometric Graph (GRGG) model.
 The tutorial is quite similar to the examples that can be found in the
 in the docstring of the `GRGG` class, but provides a somewhat more detailed
 discussion of the model. On the other hand, the docstring covers few more
 esoteric features of the package.

In [2]:
GRGG?

Init signature:
GRGG(
    n_nodes: int,
    manifold: grgg.manifolds.manifold.Manifold | int,
    *kernels: grgg.kernels.AbstractGeometricKernel,
) -> None
Docstring:     
Generalized Random Geometric Graph Model.

Attributes
----------
n_nodes
    Number of nodes in the graph.
manifold
    Manifold on which the graph is defined.
    Currently only :class:`grgg.manifolds.Sphere` is supported.
    Can also be specified as an integer, which will be interpreted
    as the surface dimension of a sphere with surface area equal to `n_nodes`.
kernels
    List of kernel functions defining the edge probabilities.
rho
    Sampling density, which is the ratio of the number of nodes
    to the surface area of the manifold.

Examples
--------
The model is initialized from a number of nodes and a manifold
on which they will live (currently only spheres are supported).

>>> from math import isclose
>>> from grgg import GRGG, Sphere, Similarity, Complementarity
>>> sphere = Sphere(2)  # unit sphere wi

## Model

The GRGG model is a random geometric graph model that generalizes the
classical random geometric graph model to allow for more complex  logic
translating between relative nodes' positions and their edge probabilities.
In particular, it allows for modeling both complementarity and similarity
driven relationships.
Like all random geometric graph, the GRGG model is defined on a metric space.
However, in order to allow for complementarity-driven relationships, here we
also assume that the metric space is compact, and asymptotically flat.
Thus, most typically the GRGG model is defined either on a (hyper)sphere or a
torus (currently only spheres are supported in the `grgg` package). In what follows,
we will always assume that nodes live on a surface of a $k$-dimensional sphere with
the surface area equal to the number of nodes. This means that the sphere is flat
in the limit $n \to \infty$.



This is how to create a GRGG model on a sphere with $n$ nodes
and surface area equal to $n$:

In [4]:
model = GRGG(100, 2) # n=100, k=2
model

GRGG(100, Sphere(2, r=2.82))

In [5]:
# Check that the surface area is correct
model.manifold.surface_area

100.00000000000001

 ### Kernels

However, in order to be useful for anything, the GRGG model has to be
equipped with one or more kernel functions that define the rules
translating between the nodes' positions and their edge probabilities.
Traditionally, random geometric graphs implicitly assume similarity kernel,
which means that the edge probability between two nodes is a decreasing function
of the distance between them. In the GRGG model, we can go beyond that and define
also complementarity kernels, which means that the edge probability
between two nodes is an increasing function of the distance between them.
But for this to make sense, the nodes have to be positioned on a compact manifold
(e.g. a sphere or a torus), so that the maximum distance between two nodes is 
well-defined and finite. Last but not least, the GRGG model can combine multiple
kernels (possibly with different strengths) to defined even more complex rules,
in particular to model both complementarity and similarity driven relationships
at the same time.

The general form of the edge probability function is given by:
$$
p_{ij} = 1 - \prod_{k=1}^K\left[
    1 - \frac{1}{1 + e^{\kappa_k(d_{ij}; \theta_k)}}
\right]
$$
where $d_{ij}$ is a geodesic distance between nodes $i$ and $j$,
$\kappa_k$ is a kernel function, $\theta_k$ is a set of parameters
of $\kappa_k$, and $K$ is the number of kernels. Importantly, the function
is equivalent to the probability that at least one of the kernels indicates that 
the edge should exist. Moreover, it can be proved that, with an appropriate 
choice of kernel functions, this form of the edge probability function is a 
solution to a physically meaningful maximum entropy problem.

#### Homogeneous kernels

Currently, we consider two types of kernels inducing homogeneous (Poissonian)
degree distributions:

* **Similarity**
  $$\kappa_{s}(d_{ij}; \mu, \beta) = \beta(d_{ij} - \mu)$$
* **Complementarity**
  $$\kappa_{c}(d_{ij}; \mu, \beta) = \beta(d_{\text{max}} - d_{ij} - \mu)$$
where $\mu$ is a distance threshold, $\beta$ controls the sharpness of the 
transition between low- and high-probability regions, and $d_{\text{max}}$ is the 
maximum between two nodes on the manifold.